# Prune Media Lists
* Drop any duplicate rows
* Drop any users with fewer than 5 item interactions

In [ ]:
import math
import os

from tqdm import tqdm

In [ ]:
media = ""

In [ ]:
outdir = "../../data/processed_data"

In [ ]:
header_fields = ""

In [ ]:
def process(remove_line, error_file):
    source = os.path.join(outdir, f"user_{media}_list.csv")
    dest = os.path.join(outdir, f"user_{media}_list.csv~")
    error_file = os.path.join(outdir, error_file)
    with open(source, "r") as in_file, open(dest, "w") as out_file, open(
        error_file, "w"
    ) as err_file:
        header = False
        for line in tqdm(in_file):
            if not header:
                header = True
                global header_fields
                header_fields = line.strip().split(",")
                out_file.write(line)
                continue
            try:
                if remove_line(line):
                    err_file.write(line)
                else:
                    out_file.write(line)
            except Exception as e:
                print(line)
                raise e
    os.rename(dest, source)

In [ ]:
def count_users(line):
    fields = line.strip().split(",")
    user_col = header_fields.index("username")
    user = fields[user_col]
    if user not in user_counts:
        user_counts[user] = 0
        user_to_uid[user] = len(user_to_uid)
    user_counts[user] += 1
    return False

In [ ]:
def remove_duplicates(line):
    fields = line.strip().split(",")
    user_col = header_fields.index("username")
    item_col = header_fields.index(f"{media}id")
    key = (user_to_uid[fields[user_col]], int(fields[item_col]))
    if key not in seen_items:
        seen_items.add(key)
        return False
    return True

In [ ]:
def remove_sparse_users(line, N=5):
    fields = line.strip().split(",")
    user_col = header_fields.index("username")
    user = fields[user_col]
    return user_counts[user] < N

In [ ]:
def remove_invalid_timestamps(line):
    fields = line.strip().split(",")
    timestamp_col = header_fields.index("timestamp")
    timestamp = fields[timestamp_col]
    return math.isclose(float(timestamp), -1)

In [ ]:
user_counts = {}
user_to_uid = {}
process(count_users, f"{media}_empty.csv")

In [ ]:
seen_items = set()
process(remove_duplicates, f"{media}_duplicates.csv")
seen_items = None

In [ ]:
process(remove_sparse_users, f"{media}_sparse_users.csv")